In [1]:
%cd ..

/home/aymeric/Documents/DAC/NeuralNetworksDIY


In [2]:
from src.convolution import *
from utils.mltools import *

def load_usps(fn):
    with open(fn, "r") as f:
        f.readline()
        data = [[float(X) for X in l.split()] for l in f if len(l.split()) > 2]
    tmp = np.array(data)
    return tmp[:, 1:], tmp[:, 0].astype(int)


def get_usps(l, X, y):
    if type(l) != list:
        resx = X[y == l, :]
        resy = y[y == l]
        return resx, resy
    tmp = list(zip(*[get_usps(i, X, y) for i in l]))
    tmpx, tmpy = np.vstack(tmp[0]), np.hstack(tmp[1])
    return tmpx, tmpy

alltrainx, alltrainy = load_usps("data/USPS_train.txt")
alltestx, alltesty = load_usps("data/USPS_test.txt")


def load_one_class(number):
    X_train, y_train = get_usps(number, alltrainx, alltrainy)
    X_test, y_test = get_usps(number, alltestx, alltesty)
    y_train = np.where(y_train == number, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == number, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


def load_two_classes(neg, pos):
    X_train, y_train = get_usps([neg, pos], alltrainx, alltrainy)
    X_test, y_test = get_usps([neg, pos], alltestx, alltesty)
    y_train = np.where(y_train == neg, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == neg, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


In [3]:
X_train, y_train, X_test, y_test = load_two_classes(6, 9)


In [6]:
alltrainx.shape

(7291, 256)

In [8]:
import numpy as np
X_train[:,:,np.newaxis].shape

(1308, 256, 1)

In [4]:
conv = Conv1D(3, 1, 1)

In [5]:
conv.forward(X_train[:,:,np.newaxis]).shape

(1308, 254, 1)

In [15]:
import torch
import torch.nn as nn


In [108]:
convtest = nn.Conv1d(3, 1, 3, stride=1, bias=False)

In [109]:
convtest.weight.data

tensor([[[ 0.2728, -0.2488, -0.1969],
         [ 0.0214, -0.2089,  0.3038],
         [ 0.1526, -0.3007,  0.0372]]])

In [104]:
S_x = np.array([[[1.,0.,-1.],[2.,0.,-2.],[1.,0.,-1.]]], dtype=float)
torch.from_numpy(S_x)

tensor([[[ 1.,  0., -1.],
         [ 2.,  0., -2.],
         [ 1.,  0., -1.]]], dtype=torch.float64)

In [105]:
convtest.weight.data = torch.from_numpy(S_x)
convtest.weight.data.shape

torch.Size([1, 3, 3])

In [110]:
input = torch.randn(20, 3, 50)
convtest(input).shape

torch.Size([20, 1, 48])

In [116]:
convtest(input)

tensor([[[ 1.7748e-01,  1.9640e-01, -3.2831e-01, -3.7376e-01,  2.9949e-01,
           1.0595e+00,  4.8277e-01, -1.2355e+00,  9.9860e-01, -2.5493e-01,
           9.9488e-03,  1.3888e+00, -7.0233e-01,  4.8568e-01, -2.5670e-01,
          -7.9850e-01, -8.5586e-02,  3.1689e-01,  2.9153e-01,  3.6671e-01,
          -6.9793e-01, -3.1381e-01,  5.8697e-01,  9.8429e-02, -5.4855e-01,
           5.1674e-01,  1.0715e-01, -2.2341e-01,  9.2742e-01, -8.2436e-02,
          -1.2197e+00,  9.8633e-01, -9.8737e-01,  2.3633e-01, -3.6227e-01,
          -3.7004e-01,  9.5992e-01,  8.5517e-01,  4.8995e-01, -1.3039e+00,
           3.1980e-01, -4.5932e-01,  2.7929e-01, -1.0391e-01,  5.5534e-01,
          -1.2520e+00,  5.8841e-02, -2.7077e-01]],

        [[ 4.2293e-01, -1.7182e-02, -6.6744e-01,  2.8552e-01, -2.2438e-01,
          -7.6975e-01, -2.2413e-01,  2.8409e-01,  1.5119e-01, -7.7447e-01,
           9.9508e-02,  1.4254e-01,  9.1906e-01, -8.1121e-01, -5.5126e-01,
           5.8236e-01,  2.1473e-01, -5.3623e-01,

In [111]:
input.numpy().shape

(20, 3, 50)

In [112]:
np.transpose(convtest.weight.data.numpy()).shape

(3, 3, 1)

In [113]:
conv = Conv1D(3, 1, 1, stride=2)
conv.parameters = np.transpose(convtest.weight.data.numpy())

In [114]:
t = conv.forward(input.numpy().reshape((20, 50, 16)))

ValueError: cannot reshape array of size 3000 into shape (20,50,16)

In [115]:
t

array([[[ 0.40629631,  0.16830748, -0.55289984, ..., -0.90844607,
         -0.22573918, -0.56154472],
        [ 0.25841379, -0.34919798, -0.44969916, ...,  0.12934335,
         -0.67064655, -0.12510088],
        [-0.48478127,  0.7343539 , -0.7678616 , ...,  1.01934981,
          0.45047149, -0.44593841],
        ...,
        [ 0.39831153,  0.19212976, -0.42594439, ..., -1.31487966,
          0.47040588,  0.00693395],
        [ 0.54658729,  0.46605903,  0.16248944, ..., -0.66372669,
         -0.3294223 ,  0.4897061 ],
        [-1.11841953, -0.37406695,  0.00343603, ..., -0.28060925,
         -0.25349227,  0.00455306]],

       [[-0.04709822, -0.03606862,  0.22745788, ...,  0.0161445 ,
         -1.07064426, -0.14689067],
        [ 0.7436806 ,  0.12690896, -0.06574279, ..., -0.24438098,
          0.65070975,  0.35783213],
        [ 0.2986739 ,  0.437379  ,  0.06449739, ..., -0.53663361,
          0.22861588,  0.10539064],
        ...,
        [ 1.06958556, -0.48588809,  0.40345812, ...,  